In [ ]:
## Required Libraries
import pandas as pd
from datasets import Dataset
import ast
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import os
import csv

/opt/miniconda3/envs/test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load FineTuned Model

In [34]:
## Import Saved Model and use with generate_playlist function
model = GPT2LMHeadModel.from_pretrained("./models/gpt2_Combined_Song_Artists", local_files_only = True)
tokenizer = GPT2Tokenizer.from_pretrained("./models/gpt2_Combined_Song_Artists", local_files_only = True)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model.to('mps') ## Use 'mps' for MacOS with M1/M2 chips
                ## Use 'cuda' for NVIDIA GPUs
                ## Use 'cpu' for CPU

## Generate Playlist Function
## This function takes a prompt and generates a playlist of songs based on the input.                
def generate_playlist(prompt, max_length=200):
    input_text = f"### Prompt: {prompt}\n### Playlist:\n"
    encoded = tokenizer(input_text, return_tensors="pt", return_attention_mask=True)
    
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    attention_mask = encoded["attention_mask"].to(model.device)

    output = model.generate(
        input_ids = input_ids.to('mps'),
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=0.9,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result.split("### Playlist:\n")[1].strip()

## Prompt Model and Generate Playlist

In [30]:
# Try it out!
prompt = "Hype Pregame"
print(generate_playlist(prompt))